In [2]:
import time
import random
import numpy as np
import networkx as nx
import logging
from itertools import combinations
import matplotlib.pyplot as plt
from icecream import ic

===========================================================================================
### Definizione del Problema
===========================================================================================

In [3]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def alpha(self):
        return self._alpha

    @property
    def beta(self):
        return self._beta

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

===========================================================================================
### Matrice delle distanze
===========================================================================================

In [4]:
def get_distance_matrix(problem:Problem) -> np.ndarray:
    graph = problem._graph
    n = len(graph.nodes)

    dist_matrix = np.zeros((n, n))
    all_pairs = nx.all_pairs_dijkstra_path_length(graph, weight="dist")

    for src, targets in all_pairs:
        for tgt, d in targets.items():
            dist_matrix[src][tgt] = d

    return dist_matrix

===========================================================================================
### Funzione di costo di una soluzione
===========================================================================================

In [5]:
def get_cost(problem: Problem, solution: list[int]) -> float:
    total_cost = 0.0
    transported_gold = 0.0

    path = [0] + solution + [0]  # start and end at depot

    empty_nodes = set() # nodes where we have picked up all the gold

    for actual_node_i in range(len(path) - 1):
        start_node = path[actual_node_i]
        destination_node = path[actual_node_i + 1]

        if start_node == 0: # if we are in the depot
            transported_gold = 0.0
        elif start_node not in empty_nodes:
            gold = problem._graph.nodes[start_node]['gold']
            transported_gold += gold
            empty_nodes.add(start_node)
            segment_cost = DISTANCE_MATRIX[start_node][destination_node] + (problem._alpha * DISTANCE_MATRIX[start_node][destination_node] * transported_gold) ** problem._beta
            total_cost += segment_cost
    return total_cost

===========================================================================================
### Funzioni a supporto dell'inizializzazione della popolazione
===========================================================================================

In [6]:
def get_initial_solution(nodes: list[int]) -> list[int]:
    unvisited_nodes = set(nodes)
    unvisited_nodes.remove(0)
    
    current_node = 0
    path = [] 
    
    while unvisited_nodes:
        candidates = []
        for node in unvisited_nodes:
            distance = DISTANCE_MATRIX[current_node][node]
            candidates.append((node, distance))
        candidates.sort(key=lambda x: x[1])
        k = min(5, len(candidates))
        top_k_candidates = candidates[:k]
        chosen_node, _ = random.choice(top_k_candidates)
        path.append(chosen_node)
        unvisited_nodes.remove(chosen_node)
        current_node = chosen_node
    return path

In [7]:
def two_opt(path: list[int], max_iterations:int) -> list[int]:
    best_path = list(path)
    length = len(best_path)
    for _ in range(max_iterations):
        i = random.randint(0, length - 2)
        j = random.randint(i + 1, length - 1)
        A = best_path[i - 1] if i > 0 else 0
        B = best_path[i]
        C = best_path[j]
        D = best_path[j + 1] if j < length - 1 else 0

        current_distance = DISTANCE_MATRIX[A][B] + DISTANCE_MATRIX[C][D]
        new_distance = DISTANCE_MATRIX[A][C] + DISTANCE_MATRIX[B][D]
        if new_distance < current_distance:
            best_path[i:j + 1] = best_path[i:j + 1][::-1]
    return best_path

def get_tsp_sequence(problem:Problem) -> list[int]:
    nodes = list(problem._graph.nodes)
    if 0 in nodes:
        nodes.remove(0)
    current_node = 0
    path = []
    unvisited_nodes = set(nodes)
    while unvisited_nodes:
        next_node = min(unvisited_nodes, key=lambda node: DISTANCE_MATRIX[current_node][node])
        path.append(next_node)
        unvisited_nodes.remove(next_node)
        current_node = next_node
    return two_opt(path, max_iterations=500)

In [8]:
def get_baseline_individual(problem:Problem) -> list[int]:
    nodes = list(problem._graph.nodes)
    if 0 in nodes:
        nodes.remove(0)
    baseline_path = []
    for node in nodes:
        baseline_path.append(node)
        baseline_path.append(0)  
    baseline_path.pop() 
    return baseline_path

def get_tsp_sequence_baseline_individual(problem:Problem)->list[int]:
    tsp_sequence = get_tsp_sequence(problem)
    baseline_path = []
    for node in tsp_sequence:
        baseline_path.append(node)
        baseline_path.append(0)  
    baseline_path.pop() 
    return baseline_path


In [9]:
def get_shuffled_baseline_individual(problem:Problem) -> list[int]:
    nodes = list(problem._graph.nodes)
    if 0 in nodes:
        nodes.remove(0)
    random.shuffle(nodes)
    path = []
    for node in nodes:
        path.append(node)
        path.append(0)
    if path and path[-1]==0:
        path.pop()
    return path

In [10]:
def get_initial_population(problem:Problem, population_size:int):
    population = []
    population.append(get_tsp_sequence_baseline_individual(problem))
    population_size = population_size-1
    while population_size!=0:
        population.append(get_shuffled_baseline_individual(problem))
        population_size = population_size-1
    return population

In [11]:
def mutation_beta_low(individual:list[int], mutation_rate:float=0.2)->list[int]:
    if random.random() > mutation_rate:
        return individual
    individual_mutated = list(individual)
    zero_indices = [i for i,x in enumerate(individual_mutated) if x==0]
    if zero_indices and random.random() < 0.9:
        zero_to_remove = random.choice(zero_indices)
        del individual_mutated[zero_to_remove]
        return individual_mutated
    else:
        length=len(individual_mutated)
        index1,index2=sorted(random.sample(range(length),2))
        individual_mutated[index1:index2+1]= individual_mutated[index1:index2+1][::-1]
        path_clean = []
        for i,node in enumerate(individual_mutated):
            if node == 0 and len(path_clean) > 0 and path_clean[-1] == 0:
                continue
            path_clean.append(node)
        if path_clean and path_clean[0] == 0:
            path_clean.pop(0)
        if path_clean and path_clean[-1] == 0:
            path_clean.pop()
        return path_clean
        


In [12]:
def order_crossover_preserving_zeroes_of_best_parent(problem:Problem, parent1:list[int],parent2:list[int])->list[int]:
    parent1_nodes = [x for x in parent1 if x!=0]
    parent2_nodes = [x for x in parent2 if x!=0]
    length = len(parent1_nodes)
    i,j = sorted(random.sample(range(length),2))
    child_nodes = [None]*length
    child_nodes[i:j+1] = parent1_nodes[i:j+1]
    index = (j+1)%length
    current_child_nodes = set(parent1_nodes[i:j+1])
    for node in parent2_nodes:
        if node not in current_child_nodes:
            while child_nodes[index] is not None:
                index = (index+1)%length
            child_nodes[index]=node
            current_child_nodes.add(node)
            index = (index+1)%length

    child = []
    node_index = 0
    if get_cost(problem, parent1) < get_cost(problem, parent2):
        best_parent=parent1
    else:
        best_parent=parent2
    for node in best_parent:
        if node == 0:
            child.append(0)
        else:
            child.append(child_nodes[node_index])
            node_index += 1
    child_clean = []
    for node in child:
        if node == 0 and child_clean and child_clean[-1] == 0:
            continue
        child_clean.append(node)
    if child_clean and child_clean[0] == 0:
        child_clean.pop(0)
    if child_clean and child_clean[-1] == 0:
        child_clean.pop()
    return child_clean

In [13]:
def selection_tournament(problem:Problem,population:list[list[int]], k=2)-> list[int]:
    competitors = random.sample(population, k)
    return min(competitors, key=lambda ind: get_cost(problem, ind))

In [14]:
def genetic_algorithm(problem: Problem, 
                      pop_size: int = 100, 
                      max_generations: int = 1000, 
                      crossover_rate: float = 0.9, 
                      mutation_rate: float = 0.2) -> tuple[list[int], float]:
    
    print(f"--- Avvio GA (Beta={problem.beta}, Elitismo=2, P_cross={crossover_rate}) ---")
    
    # --- 1. Calcolo Baseline (Riferimento) ---
    # Generiamo la soluzione "ingenua" (avanti e indietro dal deposito)
    baseline_sol = get_baseline_individual(problem)
    baseline_cost = get_cost(problem, baseline_sol)
    
    print(f"[*] Costo Baseline (Naive): {baseline_cost:.2f}")
    
    # --- 2. Inizializzazione Popolazione ---
    population = get_initial_population(problem, pop_size)
    
    # Valutazione iniziale
    pop_data = [(ind, get_cost(problem, ind)) for ind in population]
    pop_data.sort(key=lambda x: x[1]) # Ordiniamo per costo crescente
    
    best_sol, best_cost = pop_data[0]
    
    # Calcolo gap iniziale
    init_improvement = 100 * (baseline_cost - best_cost) / baseline_cost
    print(f"[*] Costo Migliore Iniziale: {best_cost:.2f} (Gap Baseline: {init_improvement:+.2f}%)")
    print("-" * 60)

    # --- 3. Ciclo Evolutivo ---
    for gen in range(1, max_generations + 1):
        
        # Elitismo (Top 2 passano intatti)
        next_gen = [item[0] for item in pop_data[:2]]
        
        current_individuals = [item[0] for item in pop_data]
        
        # Generazione Figli
        while len(next_gen) < pop_size:
            # A. Selezione
            parent1 = selection_tournament(problem, current_individuals, k=2)
            parent2 = selection_tournament(problem, current_individuals, k=2)
            
            # B. Crossover
            if random.random() < crossover_rate:
                child = order_crossover_preserving_zeroes_of_best_parent(problem, parent1, parent2)
            else:
                child = parent1[:]
            
            # C. Mutazione
            child = mutation_beta_low(child, mutation_rate)
            
            next_gen.append(child)
            
        # Aggiornamento Popolazione
        population = next_gen
        
        # Ricalcolo costi
        pop_data = [(ind, get_cost(problem, ind)) for ind in population]
        pop_data.sort(key=lambda x: x[1])
        
        # Tracking Migliore
        current_best_sol, current_best_cost = pop_data[0]
        
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_sol = list(current_best_sol)
            print(f"Gen {gen}: New Best Cost {best_cost:.2f}")
            
    # --- 4. Report Finale ---
    final_improvement = 100 * (baseline_cost - best_cost) / baseline_cost
    
    print("-" * 60)
    print(f"--- Fine GA ---")
    print(f"Costo Baseline    : {baseline_cost:.2f}")
    print(f"Costo Finale GA   : {best_cost:.2f}")
    print(f"Risparmio Assoluto: {baseline_cost - best_cost:.2f}")
    print(f"Miglioramento %   : {final_improvement:+.2f}%")
    print("-" * 60)
    
    return best_sol, best_cost

In [15]:
problem = Problem(num_cities=100, alpha=1, beta=0.5, density=0.2, seed=42)
DISTANCE_MATRIX = get_distance_matrix(problem)
genetic_algorithm(problem, 1000, 5000, 0.9, 0.3)

--- Avvio GA (Beta=0.5, Elitismo=2, P_cross=0.9) ---
[*] Costo Baseline (Naive): 1510.24
[*] Costo Migliore Iniziale: 1510.24 (Gap Baseline: +0.00%)
------------------------------------------------------------
Gen 1: New Best Cost 1507.85
Gen 2: New Best Cost 1504.62
Gen 4: New Best Cost 1504.50
Gen 5: New Best Cost 1504.50
Gen 7: New Best Cost 1502.48
Gen 12: New Best Cost 1502.48
Gen 16: New Best Cost 1501.66
Gen 25: New Best Cost 1501.23
Gen 28: New Best Cost 1501.14
Gen 30: New Best Cost 1500.54
Gen 34: New Best Cost 1500.54
Gen 41: New Best Cost 1500.54
Gen 58: New Best Cost 1500.15
Gen 80: New Best Cost 1499.83
Gen 210: New Best Cost 1498.98
Gen 218: New Best Cost 1498.04
Gen 239: New Best Cost 1494.65
Gen 266: New Best Cost 1493.81
Gen 309: New Best Cost 1493.18
Gen 311: New Best Cost 1492.71
Gen 316: New Best Cost 1491.86
Gen 336: New Best Cost 1491.18
Gen 360: New Best Cost 1490.64
Gen 441: New Best Cost 1486.54
Gen 457: New Best Cost 1486.28


KeyboardInterrupt: 